<h1 style="text-align:center;">Setting the UID light</h1>


<div class="alert alert-block alert-info"  style="border-radius: 20px;">
Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>Setting UID light</b><br>
    This Jupyter Notebook will contains a example code to demonstrate how to set uid light. This code is written in Python and uses the redfish library to perform the required operations.</b><br> As you are here let's get it done easily.<br>
    To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    Please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary modules from the ilorest library.</h3><br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
        
</div>

In [ ]:
import sys
import json
import time
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory
import getpass

<div class = "alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 2. This is our set_uid_light to function overview, in the end this will set uid light.</h3><br>
    
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.<br>
1. This is a function that uses the Redfish API to toggle the UID (Universal Identification) LED on a computer system.
    The function takes a Redfish connection object <b>_redfishobj</b> as its only argument.<br><br>
2. In the <b>set_uid_light</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br><br>
3.  In the iLO rest api the information about UID light is stored inside the <b>systems json object</b>.
    And now through system_response variable, we are making a http <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods" style="background-color:powderblue;">GET operation</a> to and trying to obtain the UID light settings of iLO system.
    <br><br>
4. In the system members response object with different <b>odata_id</b> we get a system uri . Then again an http GET request is made and response is stored in <b>system_response</b>. Then we set the <b>system_members_uri<b>.<br><br>
5. Else if resource directory is enabled then we will simply iterate through all instances inside resources and if #ComputerSystem attribute is present in <b>@odata.type</b> attribute then we'll set <b>system_members_uri</b> and <i> get </i> <b>system_members_response settings</b> of the user through redfish object.<br><br>
6.It sets the <b>body["IndicatorLED"]</b> key to <b>Lit</b> if the current value of the IndicatorLED property is <b>Off</b>, or to "Off" otherwise.<br><br>
7. It sends an HTTP PATCH request to the obtained URI with the body payload to update the IndicatorLED property.<br><br>
8. If the response status code is not 200 (OK) or 201 (Created), it prints an error message and exits.<br><br>
9. If the response status code is 200 or 201, it waits for 10 seconds to allow the firmware to update the LED status, and then sends a GET request to the same URI to obtain the updated resource representation of the computer system.<br><br>
10. It prints the updated value of the IndicatorLED property.<br><br>
 </div>

In [ ]:
def set_uid_light(_redfishobj):

    body = dict()
    systems_members_uri = None
    systems_members_response = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#ComputerSystem.' in instance['@odata.type']:
                systems_members_uri = instance['@odata.id']
                systems_members_response = _redfishobj.get(systems_members_uri)

    if systems_members_response and systems_members_uri:
        print("Current Indicator LED Status: \'%s\'\n" % systems_members_response.dict.\
                                                                            get("IndicatorLED"))
        if "Off" in systems_members_response.dict.get("IndicatorLED"):
            print("Will illuminate indicator LED.\n")
            body["IndicatorLED"] = "Lit"
        else:
            print("Will extinguish indicator LED.\n")
            body["IndicatorLED"] = "Off"

        resp = _redfishobj.patch(systems_members_uri, body)
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
            time.sleep(10) #going to wait 10 seconds before obtaining the LED indicator state
            sys.stdout.write("\nUpdated Indicator LED Status: \'%s\'\n" % _redfishobj.\
                                                    get(systems_members_uri).dict['IndicatorLED'])



<div class = "alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 3. This is Main function overview.<br></h3>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.<br>

1. The rest opertions are handled by Redfish object which is created by Redfish Client<br>
<br>
    
2. <b>Redfish Client</b> class takes 3 parameters<i> iLO url, Username, Password</i> and returns a redfish object.<br>
    <br>

3. Once you enter everything a redfish object is created with the help of Redfish Client class.<br>
    <br>

4. Now we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful a session is created and you are logged in, But if Server is not avaialbe we get an error saying "server unreachable"<br>
    <br>
5. Then we call <b>set_uid_light</b> with our redfish object to set UID light.<br>
    <br>
6. <b>REDFISHOBJ.logout()</b> will take care of logging you out of the ilo.

 </div>

<div class="alert alert-block alert-warning"  style="border-radius: 20px;">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":

    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD =getpass.getpass("Enter password: ")

    # flag to force disable resource directory. Resource directory and associated operations are intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_uid_light(REDFISHOBJ)
    REDFISHOBJ.logout()


<div class = "alert alert-block alert-success"  style="border-radius: 20px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>